In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import json
import re
import nltk
import tkinter as tk
from tkinter import ttk
import ast

In [2]:
movies_df = pd.read_csv("source_data/movies_metadata.csv")


C:\Users\chanc\AppData\Local\Temp\ipykernel_12652\721637515.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("source_data/movies_metadata.csv")


In [3]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
# # Initialize the main window
# root = tk.Tk()
# root.title("Movie Recommender")

# # Define the function to show recommendations
# def show_recommendations():
#     genre = genre_input.get()
#     year = year_input.get()
#     rating = rating_input.get()
#     recommendations = recommend_movies(genre, year, rating)
#     result_var.set("\n".join(recommendations))

# # Simple recommendation function considering multiple inputs
# def recommend_movies(genre, year, rating):
#     filtered_movies = movies_df
    
#     if genre:
#         filtered_movies = filtered_movies[filtered_movies['genres'].str.contains(genre, case=False, na=False)]
#     if year:
#         filtered_movies = filtered_movies[filtered_movies['release_date'].str.contains(year, na=False)]
#     if rating:
#         filtered_movies = filtered_movies[filtered_movies['vote_average'] >= float(rating)]
    
#     top_movies = filtered_movies.head(10)['title'].tolist()
#     return top_movies

# # Add a label and entry for genre input
# ttk.Label(root, text="Enter your preferred genre:").grid(column=0, row=0, padx=10, pady=10)
# genre_input = ttk.Entry(root)
# genre_input.grid(column=1, row=0, padx=10, pady=10)

# # Add a label and entry for year input
# ttk.Label(root, text="Enter release year:").grid(column=0, row=1, padx=10, pady=10)
# year_input = ttk.Entry(root)
# year_input.grid(column=1, row=1, padx=10, pady=10)

# # Add a label and entry for rating input
# ttk.Label(root, text="Enter minimum rating:").grid(column=0, row=2, padx=10, pady=10)
# rating_input = ttk.Entry(root)
# rating_input.grid(column=1, row=2, padx=10, pady=10)

# # Add a button to trigger recommendation
# ttk.Button(root, text="Get Recommendations", command=show_recommendations).grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# # Add a label to display the results
# result_var = tk.StringVar()
# ttk.Label(root, textvariable=result_var).grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# # Run the application
# root.mainloop()


In [4]:
# Define a function to extract genres from a single row entry
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)  # Convert string to list of dicts
        return [genre['name'] for genre in genres]
    except:
        return []

# Use the function to extract genres for the entire column
all_genres = set()
movies_df['genres'].dropna().apply(lambda x: all_genres.update(extract_genres(x)))

# Convert the set to a sorted list
all_genres = sorted(list(all_genres))

# Display all unique genres
print(all_genres)

['Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV', 'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror', 'Mardock Scramble Production Committee', 'Music', 'Mystery', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance', 'Science Fiction', 'Sentai Filmworks', 'TV Movie', 'Telescene Film Group Productions', 'The Cartel', 'Thriller', 'Vision View Entertainment', 'War', 'Western']


In [5]:
# Search for movies produced by "Carousel Productions"
carousel_movies = movies_df[movies_df['production_companies'].str.contains("Carousel Productions", case=False, na=False)]

# Display the titles of the movies
carousel_movie_titles = carousel_movies['title'].tolist()
carousel_movie_titles


['The Deadly Companions', 'The Incredible Burt Wonderstone']

In [7]:
# Extract unique genres from the dataset
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [genre['name'] for genre in genres]
    except:
        return []

# Extract the release year from the release_date column
movies_df['release_year'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year

all_genres = set()
all_release_years = sorted(movies_df['release_year'].dropna().astype(int).unique(), reverse=True)

movies_df['genres'].dropna().apply(lambda x: all_genres.update(extract_genres(x)))

all_genres = sorted(list(all_genres))

# Initialize the main window
root = tk.Tk()
root.title("Movie Recommender")

# Define the function to show recommendations
def show_recommendations():
    genre = genre_input.get()
    release_year = release_year_input.get()
    keywords = keyword_input.get()
    recommendations = recommend_movies(genre, release_year, keywords)
    result_var.set("\n".join(recommendations))

# Calculate match percentage based on genre, release year, and keywords
def calculate_match_percentage(row, genre, release_year, keywords):
    match_score = 0
    total_criteria = 3  # We have three criteria now: genre, release year, and keywords
    
    # Check genre match
    if genre and genre != "Any":
        if genre in extract_genres(row['genres']):
            match_score += 1
    
    # Check release year match
    if release_year and release_year != "Any":
        if pd.notna(row['release_year']) and row['release_year'] == int(release_year):
            match_score += 1
            
    # Check keyword match
    if keywords:
        if pd.notna(row['overview']) and keywords.lower() in row['overview'].lower():
            match_score += 1
    
    # Calculate percentage
    return (match_score / total_criteria) * 100

# Recommendation function with percentage calculation
def recommend_movies(genre, release_year, keywords):
    filtered_movies = movies_df
    
    if genre and genre != "Any":
        filtered_movies = filtered_movies[filtered_movies['genres'].str.contains(genre, case=False, na=False)]
    if release_year and release_year != "Any":
        filtered_movies = filtered_movies[filtered_movies['release_year'] == int(release_year)]
    if keywords:
        filtered_movies = filtered_movies[filtered_movies['overview'].str.contains(keywords, case=False, na=False)]
    
    # Calculate match percentages
    filtered_movies['match_percentage'] = filtered_movies.apply(lambda row: calculate_match_percentage(row, genre, release_year, keywords), axis=1)
    
    # Sort by match percentage and limit to top 5
    filtered_movies = filtered_movies.sort_values(by='match_percentage', ascending=False)
    top_movies = filtered_movies.head(5)[['title', 'match_percentage']]
    
    # Format the output
    result = [f"{row['title']} - {row['match_percentage']:.2f}%" for _, row in top_movies.iterrows()]
    return result

# Add a label and dropdown for genre selection
ttk.Label(root, text="Select a genre:").grid(column=0, row=0, padx=10, pady=10)
genre_input = ttk.Combobox(root, values=["Any"] + all_genres)
genre_input.grid(column=1, row=0, padx=10, pady=10)
genre_input.set("Any")

# Add a label and dropdown for release year selection
ttk.Label(root, text="Select a release year:").grid(column=0, row=1, padx=10, pady=10)
release_year_input = ttk.Combobox(root, values=["Any"] + [str(year) for year in all_release_years])
release_year_input.grid(column=1, row=1, padx=10, pady=10)
release_year_input.set("Any")

# Add a label and entry for keyword input
ttk.Label(root, text="Enter keywords:").grid(column=0, row=2, padx=10, pady=10)
keyword_input = ttk.Entry(root)
keyword_input.grid(column=1, row=2, padx=10, pady=10)

# Add a button to trigger recommendation
ttk.Button(root, text="Get Recommendations", command=show_recommendations).grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# Add a label to display the results
result_var = tk.StringVar()
ttk.Label(root, textvariable=result_var).grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()